# Build / Use Vector Store
Below is a simple example of first building an in memory vector store from a Notion markdown text directory, and then using the store to query for search results.

## Env Setup
Before we get started, install the gems we need here:
`gem install boxcars dotenv`
and then create / edit .env to have OPENAI_ACCESS_TOKEN

In [1]:
require 'dotenv/load'
require 'boxcars'

true

## Build the Vector Store

In [2]:
root = "./embeddings"
store = Boxcars::VectorStore::Hnswlib::BuildFromFiles.call(
  training_data_path: "#{root}/Notion_DB/**/*.md",
  index_file_path: "#{root}/hnswlib_notion_db_index.bin",
  force_rebuild: true
)
puts :built

Building Hnswlib vector store...
Added 50 files to data. Splitting text into chunks...
Loaded 50 files from /Users/francis/src/notebooks/boxcars/embeddings/Notion_DB/**/*.md
Split 50 files into 140 chunks
Generated 140 vectors
Added 140 vectors to vector store
built


## Query the Store

In [3]:
similarity_search = Boxcars::VectorSearch.new(
  embeddings: "#{root}/hnswlib_notion_db_index.json",
  vector_documents: store)
ss = similarity_search.call query: "Do I get a laptop?", count: 1
ss.first[:document].content

"we provide you with a laptop that suits your job. Ask HR for further info.\n- **Workplace**: \nwe've built a pretty nice office to make sure you like being at Blendle HQ. Feel free to sit where you want. Even better: dare to switch your workplace every once in a while.\n\n# Work at Blendle\n\n---\n\nIf you want to work at Blendle you can check our [job ads here](https://blendle.homerun.co/). If you want to be kept in the loop about Blendle, you can sign up for [our behind the scenes newsletter](https://blendle.homerun.co/yes-keep-me-posted/tr/apply?token=8092d4128c306003d97dd3821bad06f2)."

## Answer a Question from Search Results

In [7]:
va = Boxcars::VectorAnswer.new(embeddings: "#{root}/hnswlib_notion_db_index.json", vector_documents: store)
va.conduct("Do I get a laptop?").to_answer

> Entering VectorAnswer#run
Do I get a laptop?
{"status":"ok","answer":"Yes, you will be provided with a laptop that suits your job. You can ask HR for further information.","explanation":"Answer: Yes, you will be provided with a laptop that suits your job. You can ask HR for further information."}
< Exiting VectorAnswer#run


"Yes, you will be provided with a laptop that suits your job. You can ask HR for further information."

## More
You could of course use text files and get similar results. Other libraries can be brought it to handle PDFs and other binary formats. Add Issues and/or PRs for other types that you want supported.